In [1]:
# Data Analysis and Wrangling 
import pandas as pd 
import numpy as np
import random as rnd

# Visualization 
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline 

# Machine Learning  
from sklearn.linear_model import LogisticRegression 
# from sklearn.svm import SVC, LinearSVC
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.linear_model import Perceptron 
# from sklearn.linear_model import SGDClassifier
# from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics 
from sklearn.cross_validation import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
path1 = '../Cleaned-Data/Training Data.xlsx'
path2 = '../Cleaned-Data/Testing Data.xlsx'

In [3]:
df_training = pd.read_excel(path1)
df_testing = pd.read_excel(path2)

In [4]:
df_testing.head()

,Average GPA,Coding,Logical
0,1.5200,80.03,33.53
1,3.4900,64.47,49.67
2,5.4475,64.60,33.40
3,5.4100,50.76,67.41
4,9.0400,24.53,86.73


### Create a function for classification model

In [5]:
def classification_model(model, data, predictors, outcome): 
    # Fit the model
    model.fit(data[predictors], data[outcome])
    
    # Make predictions on training set: 
    predictions = model.predict(data[predictors])
    
    #print accuracy 
    accuracy = metrics.accuracy_score(predictions, data[outcome])
    print "Accuracy : %s" % '{0: .3%}'.format(accuracy)
    
    # Perform k-fold validation with 5 folds
    kf = KFold(data.shape[0], n_folds=5)
    error = []
    for train, test in kf:
        # Filter training data
        train_predictors = (data[predictors].iloc[train,:])
        
        # The target we're using to train the algorithm
        train_target = data[outcome].iloc[train]
        
        # Training the algorithm using the predictors and target.
        model.fit(train_predictors, train_target)
        
        # Record error from each cross-validation run
        error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))
        
    print 'Cross-Validation Score: %s' % '{0: .3%}'.format(np.mean(error))
        
    # Fit the model again so that it can be refered outside the function:
    model.fit(data[predictors], data[outcome])

In [6]:
outcome_var = 'Placed 1/0'
model = LogisticRegression()
predictor_var = ['Average GPA','Coding','Logical']
classification_model(model, df_training, predictor_var, outcome_var)

Accuracy :  66.667%
Cross-Validation Score:  65.608%


### ------------- Yet to work on above part --------------

In [7]:
train = df_training.drop('Placed 1/0', axis=1)
# Y_train = df_training['Placed 1/0']
# X_test = df_testing.copy()
# X_train.shape, Y_train.shape, X_test.shape

## Artifical Neural Network

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, df_training['Placed 1/0'])
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((211, 3), (211,), (71, 3), (71,))

In [9]:
scaler = StandardScaler()     # For standardization
# Fit only to the training data
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [10]:
# Now apply the transformations to the data: 
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
# Number shows the number of neurons in the nth layer of the MLP model
# For simplicity keep this as the same number of neurons as there are features in the dataset
mlp = MLPClassifier(hidden_layer_sizes=(3,3,3))

In [13]:
mlp.fit(X_train, y_train)

/anaconda/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(3, 3, 3), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [14]:
predictions = mlp.predict(X_test)

In [15]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, predictions))

[[34  0]
 [37  0]]


In [16]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.48      1.00      0.65        34
          1       0.00      0.00      0.00        37

avg / total       0.23      0.48      0.31        71



/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


ANN gives lower precision value compared to other algorithms because of size of dataset. This proves ML is useful for smaller datasets but for larger datasets ANN is useful as it improves over time with larger dataset. 